https://keras.io/guides/writing_a_training_loop_from_scratch/

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Activation ,AveragePooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc,precision_score, recall_score,f1_score,accuracy_score
from sklearn.model_selection import StratifiedKFold
from scipy import interp
import matplotlib.pyplot as plt
from skimage import transform
import scipy.ndimage
import matplotlib.pyplot as plt

from keras.layers import Lambda, Input
from keras.models import Model, Sequential
import tensorflow as ktf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('Train data shape: {}'.format(X_train.shape))
print('Test  data shape: {}'.format(X_test.shape))

Train data shape: (50000, 32, 32, 3)
Test  data shape: (10000, 32, 32, 3)


In [ ]:
image_size = 32
num_channels = 3
num_features = image_size * image_size * num_channels
num_classes = 10

In [ ]:
X_train = X_train.astype('float32')
x_test  = X_test.astype('float32')

X_train /= 255.0
mean = np.mean(X_train, axis=(0, 1, 2,3))
std = np.std(X_train, axis=(0, 1, 2,3))
X_train = (X_train - mean)/ (std+1e-7)

x_test /= 255.0

x_test =(x_test-mean) / (std+1e-7)

print(X_train.shape)
print(x_test.shape)


(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test,  num_classes)

print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


In [ ]:
# plotting helper function
def plothist(hist):
    
    plt.subplot(1, 2, 1)
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')

    plt.subplot(1, 2, 2)
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')


    plt.show()

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-fxxqtgzq
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-fxxqtgzq
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=20030 sha256=54543779ddbb4ce21fd23a2a75c3f9eec59cd4e14798e95e1744f4e45276041c
  Stored in directory: /tmp/pip-ephem-wheel-cache-5wjjan0g/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [ ]:
from classification_models.tfkeras import Classifiers

In [ ]:
ResNet18, preprocess_input = Classifiers.get('resnet18')
resnet18 = ResNet18((224, 224, 3), weights='imagenet')
resnet18.summary() 

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
______________________________________________________________________________________________

In [ ]:
alpha = 0.003  # weight decay coefficient

for layer in resnet18.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
        layer.add_loss(lambda: keras.regularizers.l2(alpha)(layer.kernel))
    if hasattr(layer, 'bias_regularizer') and layer.use_bias:
        layer.add_loss(lambda: keras.regularizers.l2(alpha)(layer.bias))

In [ ]:
x = resnet18.layers[-2].output
output = Dense(units=10,activation='softmax')(x)
model = Model(inputs=[resnet18.input], outputs=[output])
#model.summary()

In [ ]:
for layer in model.layers[75:]:
    layer.trainable = True
for layer in model.layers[:75]:
    layer.trainable = False
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 data False
1 bn_data False
2 zero_padding2d False
3 conv0 False
4 bn0 False
5 relu0 False
6 zero_padding2d_1 False
7 pooling0 False
8 stage1_unit1_bn1 False
9 stage1_unit1_relu1 False
10 zero_padding2d_2 False
11 stage1_unit1_conv1 False
12 stage1_unit1_bn2 False
13 stage1_unit1_relu2 False
14 zero_padding2d_3 False
15 stage1_unit1_conv2 False
16 stage1_unit1_sc False
17 add False
18 stage1_unit2_bn1 False
19 stage1_unit2_relu1 False
20 zero_padding2d_4 False
21 stage1_unit2_conv1 False
22 stage1_unit2_bn2 False
23 stage1_unit2_relu2 False
24 zero_padding2d_5 False
25 stage1_unit2_conv2 False
26 add_1 False
27 stage2_unit1_bn1 False
28 stage2_unit1_relu1 False
29 zero_padding2d_6 False
30 stage2_unit1_conv1 False
31 stage2_unit1_bn2 False
32 stage2_unit1_relu2 False
33 zero_padding2d_7 False
34 stage2_unit1_conv2 False
35 stage2_unit1_sc False
36 add_2 False
37 stage2_unit2_bn1 False
38 stage2_unit2_relu1 False
39 zero_padding2d_8 False
40 stage2_unit2_conv1 False
41 stage2_unit2_bn2

In [ ]:
newInput = Input(batch_shape=(None, 32, 32, 3))
resizedImg = Lambda(lambda image: ktf.image.resize(image, (224, 224)))(newInput)
newOutputs = model(resizedImg)
model = Model(newInput, newOutputs)

In [ ]:
BATCH_SIZE =100
STEPS_PER_EPOCH =len(X_train)//BATCH_SIZE 

#step = tf.Variable(0, trainable=False)
boundaries = [30*STEPS_PER_EPOCH,50*STEPS_PER_EPOCH,80*STEPS_PER_EPOCH]
values = [ 0.1, 0.01,0.001,0.0001]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)

In [ ]:
opt = SGD(learning_rate=learning_rate_fn)#,momentum=0.9,decay=0.01,nesterov=False
#opt = Adam(learning_rate=0.0001,beta_1=0.9,beta_2=0.99,epsilon=0.1,amsgrad=False,name="Adam",)
#model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

In [ ]:
x_train = np.reshape(X_train, (-1, 1024))
x_test = np.reshape(x_test, (-1, 1024))

In [ ]:
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)


In [ ]:
# Instantiate an optimizer.
optimizer = opt
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 64))


Start of epoch 0


ValueError: ignored

In [ ]:
y_pred = model.predict(x_test)
labels1 = np.arange(num_classes)
print('Auc = %f' %  (roc_auc_score(y_test,  y_pred.round(),average='macro',multi_class='ovo',labels=labels1)))
print('f1_score = %f' % (f1_score(y_test, y_pred.round(), average="macro")))
print('precision = %f' % (precision_score(y_test, y_pred.round(), average="macro")))
print('recall  = %f' % (recall_score(y_test, y_pred.round(), average="macro")))
print('accuracy  = %f' % (accuracy_score(y_test, y_pred.round()))) 

In [ ]:
plothist(history)